
# Interactive API Fundamentals Presentation

Welcome to this interactive slideshow on **Using APIs in Python**! 🚀

### What you will learn:
- Basics of APIs and JSON data
- How to make API requests in Python
- Using authentication (API keys, application tokens)
- Converting API data into GeoPandas
- Visualizing geospatial data



## API Fundamentals in Python


### Introduction

APIs (Application Programming Interfaces) allow different software applications to communicate with each other.
They provide a structured way for applications to request and exchange data over the internet.

**Why use APIs?**
- Access real-time data (weather, stock prices, etc.).
- Automate tasks and integrate with other platforms.
- Retrieve data from databases and services efficiently.

Let's explore the fundamentals of APIs in Python using the `requests` library and `sodapy`.


In [5]:

# !pip install sodapy
# !pip install shapely
%pip install riverrem
# !pip install geopandas

Note: you may need to restart the kernel to use updated packages.


You can import the environment file using this line: 
conda env create --name UDA_Imported -f D:\Projects\MyProject\my_env.yml

In [ ]:
import requests
import json
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd

ModuleNotFoundError: No module named 'gdal'


### Understanding JSON Format

APIs often return data in JSON (JavaScript Object Notation) format. JSON is a lightweight data-interchange format
that is easy for humans to read and write and easy for machines to parse and generate.

#### JSON Structure
JSON consists of key-value pairs, similar to a Python dictionary. Example:

```json
{
    "id": 1,
    "title": "Introduction to APIs",
    "author": "John Doe",
    "tags": ["API", "Python", "JSON"]
}
```

- Strings are enclosed in double quotes.
- Values can be strings, numbers, booleans, arrays (lists), or nested objects (dictionaries).

#### Parsing JSON in Python
In Python, the `json` module and `requests` library help us handle JSON data:


In [7]:

# JSON string
json_data = '{"id": 1, "title": "Introduction to APIs", "author": "John Doe"}'

# Parsing JSON string to a Python dictionary
parsed_data = json.loads(json_data)
print(parsed_data["title"])  # Output: Introduction to APIs

# Converting Python dictionary back to JSON string
json_string = json.dumps(parsed_data, indent=4)
print(json_string)


Introduction to APIs
{
    "id": 1,
    "title": "Introduction to APIs",
    "author": "John Doe"
}



### Understanding API Endpoints and Parameters

APIs typically have endpoints, which are specific URLs that provide access to resources.
When making API requests, we often pass parameters to filter and refine the data.

Example API request structure:
```
https://api.example.com/data?key=value&param=value
```
- `https://api.example.com/data` -> The API endpoint
- `?key=value&param=value` -> Query parameters to modify the request

#### Using Parameters in API Requests
Many APIs allow you to send parameters to customize the response. Let's look at a few examples.

**Example 1: Filtering Posts by User ID**


In [8]:

params = {"userId": 1}  # Filtering posts by user ID
url = "https://jsonplaceholder.typicode.com/posts"
response = requests.get(url, params=params)

if response.status_code == 200:
    posts = response.json()
    print("First Post for User 1:", posts[0])  # Displaying first result
else:
    print("Error fetching data.")


First Post for User 1: {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}



**Example 2: Fetching Weather Data Using Parameters**

We'll use Open-Meteo's free weather API to fetch real-time weather data.


In [9]:

weather_url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 40.7128,  # New York City
    "longitude": -74.0060,
    "current_weather": True
}

response = requests.get(weather_url, params=params)

if response.status_code == 200:
    weather_data = response.json()
    print("Current Weather in NYC:", weather_data["current_weather"])
else:
    print("Failed to retrieve weather data.")


Current Weather in NYC: {'time': '2025-08-05T21:00', 'interval': 900, 'temperature': 27.2, 'windspeed': 17.9, 'winddirection': 124, 'is_day': 1, 'weathercode': 3}



### Understanding API Keys and Tokens

Some APIs require authentication using API keys or tokens. An API token is a unique identifier used to authenticate requests to the API provider.

#### What Are API Tokens?
- API tokens serve as a way to authenticate and authorize users.
- They help in monitoring API usage and enforcing rate limits.
- Some APIs work without a token but have limitations on the number of requests.

#### Using an API Token
When an API requires a token, you usually pass it as a query parameter or in the request headers.

Example (using a fictional API token for demonstration):



#### Using the NYC Open Data API With and Without a Token

The NYC Open Data API allows access to public datasets with and without an app token. Using a token provides higher rate limits and access to additional data.

Without a token:


In [ ]:
# https://dev.socrata.com/foundry/data.cityofnewyork.us/h9gi-nx95
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("h9gi-nx95", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


With a token:


In [ ]:

api_token = "---------------"
params = {"$limit": 5, "$$app_token": api_token}
response = requests.get(api_endpoint, params=params)

if response.status_code == 200:
    print("Response with token:", response.json())
else:
    print("Error:", response.status_code)


In [ ]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime

# 1. Get your app token from:
# https://dev.socrata.com/docs/app-tokens.html
APP_TOKEN = "------------------------"  # Replace with your actual token

# 2. Configure date range (YYYY-MM-DD)
start_date = "2023-01-01"
end_date = "2023-12-31"

# Initialize authenticated client
client = Socrata(
    "data.cityofnewyork.us",
    APP_TOKEN,
    # Optional for public data, required for private datasets:
    # username="your_email@example.com",
    # password="your_password"
)

# Build date filter query
date_filter = f"crash_date >= '{start_date}T00:00:00' AND crash_date <= '{end_date}T23:59:59'"

# Paginated download with app token benefits
results = []
limit = 200  # Max allowed with token (vs 1000 without)
offset = 0

while True:
    batch = client.get(
        "h9gi-nx95",
        where=date_filter,
        limit=limit,
        offset=offset
    )
    if not batch:
        break
    results.extend(batch)
    offset += limit
    print(f"Downloaded {len(results)} records...")

# Convert to DataFrame with proper typing
df = pd.DataFrame.from_records(results).pipe(
    lambda d: d.assign(
        crash_date=pd.to_datetime(d['crash_date']),
        latitude=pd.to_numeric(d['latitude']),
        longitude=pd.to_numeric(d['longitude'])
    )
)

print(f"\nSuccessfully retrieved {len(df)} crashes between {start_date} and {end_date}")


In [ ]:
df.sample()

In [ ]:
df = df.loc[(df["latitude"]!=0) & (df["longitude"]!=0)]

In [ ]:


# Create a GeoDataFrame with points
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.sample()


In [ ]:
# Plot map
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, markersize=5, color="blue", alpha=0.5)
plt.title("NYC Motor Vehicle Collisions")
plt.xlabel("Longitude")
plt.ylabel("Latitude")